In [8]:
import re
import monpa
import pandas as pd
def get_dataset(filepath):
    sentence = []
    ID = []
    with open(filepath,'r',encoding='UTF-8') as f:
        for line in f.read().splitlines():
            items = line.split('\t')   #用tab隔開
            if(len(items)==2):
                sentence.append(items[0])
                ID.append(items[1])
    return sentence ,ID
label ,sentence=get_dataset('./E-commerce reviews_chinese.txt')

def preprocessing(sentence):
    final_data = []
    result_cut_batch = []
    final_sent = []
    final_sentence = []
    len_sent = []
    for i in sentence:
        a = re.sub(r'[^\u4e00-\u9fa5]','',i)
        final_data.append(a)    
    monpa.load_userdict("./userdict1.txt")
    for i in final_data:
        item = monpa.cut(i)
        result_cut_batch.append(item)  
    stopwords_data = pd.read_csv("./NEW_stopwords.csv",encoding = 'utf-8')
    stopWords = list(stopwords_data["stopwords"])    
    for i in result_cut_batch:
        items = list(filter(lambda a: a not in stopWords and a != '\n', i))
        final_sent.append(items)
    for i in final_sent:
        a = len(i)
        len_sent.append(a)
    for i in range(len(final_sent)):
        if len_sent[i] == 0:
            yuya= result_cut_batch[i]
        else:
            yuya=final_sent[i]
        final_sentence.append(yuya)
    return final_sentence
final_sentence = preprocessing(sentence)

In [51]:
import os
from keras_bert import load_trained_model_from_checkpoint
import numpy as np
import codecs
from keras_bert import Tokenizer
MAX_SEQ_LEN = 30
BERT_PRETRAINED_DIR = './chinese_L-12_H-768_A-12' # need to rewrite if you put model in other directory
config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, seq_len=MAX_SEQ_LEN)
model.summary(line_length=120)
def get_bert_embedding2 (word):
    dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
    token_dict = {}
    with codecs.open(dict_path, 'r', 'utf8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)
       
    tokenizer = Tokenizer(token_dict)
    bert=[]
    for i in word:
        tokens = tokenizer.tokenize(i)
    #     print('Tokens:', tokens)
    #     print('==============================================================')
        indices, segments = tokenizer.encode(first=i, max_len=MAX_SEQ_LEN)
    # extract vectors
        predicts = model.predict([np.array([indices]), np.array([segments])])[0]
        bert.append(predicts[0].tolist()) # 僅列出前5個向量值
    return bert
total_bert = get_bert_embedding2(sentence)

Model: "model_4"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, 30)                 0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, 30)                 0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 30, 768), (21128,  16226304      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segme

In [94]:
import heapq
"""
total_V_ans and total_A_ans are the Valence and Arousal of each word
Please refer to page 7 in our paper to know the detail for calculating the Valence and Arousal.
"""

def top3VA(final_sentence,final_LLRsentence,total_V_ans,total_A_ans,total_dict):
    total_word = []
    nnV = []
    nnV1 = []
    for i in final_sentence:
        for item in i:
            total_word.append(item)
    total_word = to_total(final_sentence)
    total_V = to_total(total_V_ans)
    total_A = to_total(total_A_ans)
    totalwordV_dict = dict(zip(total_word,total_V))
    totalwordA_dict = dict(zip(total_word,total_A))
    for i in total_dict.keys():
        for item in totalwordV_dict:
            if i == item:
                nnV.append(totalwordV_dict[item])
                nnV1.append(i)
    total_dictvalue = []
    for i in nnV1:
        a = total_dict.get(i)
        total_dictvalue.append(a)
    nnA = []
    for i in total_dict.keys():
        for item in totalwordA_dict:
            if i == item:
                nnA.append(totalwordA_dict[item])
    LLR_dictVA = list(zip(nnV1,total_dictvalue,nnV,nnA))
    original_ssent = list(zip(total_word,total_V,total_A))
    top3VA = []
    avvv = []
    for i in range(len(final_LLRsentence)):
        for u in final_LLRsentence[i]:
            xx =[]
            for item in LLR_dictVA:
                if len(final_LLRsentence[i])>3:
                    if u == item[0]:
                        avvv.append(item[1])
                        a = heapq.nlargest(3,avvv)
                        for ig in a:
                            b = [y[1] for y in LLR_dictVA].index(ig)
                            final_v = LLR_dictVA[b][2]
                            final_v1 = LLR_dictVA[b][3]
                            xx.append([final_v,final_v1])
                elif len(final_LLRsentence[i])==3:
                    if u == item[0]:
                        for ig in final_LLRsentence[i]:
                            b = [y[0] for y in LLR_dictVA].index(ig)
                            final_v = LLR_dictVA[b][2]
                            final_v1 = LLR_dictVA[b][3]
                            xx.append([final_v,final_v1])
                elif len(final_LLRsentence[i])<3: 
                    if u == item[0]:
                        for ig in final_LLRsentence[i]:
                            b = [y[0] for y in LLR_dictVA].index(ig)
                            final_v = LLR_dictVA[b][2]
                            final_v1 = LLR_dictVA[b][3]
                            xx.append([final_v,final_v1])
                        if final_sentence[i] != item[0]:                            
                            k = heapq.nlargest(3-len(final_LLRsentence[i]),total_V_ans[i])
                            for ll in k:
                                b1 = [y[1] for y in original_ssent].index(ll)
                                final_a = original_ssent[b1][1]
                                final_a1 = original_ssent[b1][2]
                                xx.append([final_a,final_a1])
        top3VA.append(xx)
    final_top3va = []
    final_top3VA = []
    for i in top3VA:
        a = []
        for item in i:
            for items in item:
                a.append(items)
        final_top3va.append(a)
    for i in final_top3va:
        b = list(i + [0] * (6 - len(i)))
        final_top3VA.append(b)
    return final_top3VA
final_top3VA = top3VA(final_sentence,final_LLRsentence,total_V_ans,total_A_ans,total_dict)

In [ ]:
import numpy as np
def get_sentiment_key_word():
    """
    In order to better identify the sentiment of the messages, 
    Log-Likelihood Ratio (LLR) estimation is introduced in this research 
    to score the most representative words for the positive and negative categories. 
    """
    return pos_keyword,neg_keyword

pos_keyword,neg_keyword = get_sentiment_key_word()
total_dict = dict(pos_keyword, **neg_keyword)
def ComparisonLLR(final_sentence):
    final_LLRsentence = []
    for i in final_sentence:
        a = []
        for item in i:
            if item in total_dict:
                a.append(item)
        final_LLRsentence.append(a)
    return final_LLRsentence
final_LLRsentence = ComparisonLLR(final_sentence)
def LLR200(final_sentence,pos_keyword,neg_keyword):
    pos_nn = []
    neg_nn = []
    for item in final_sentence:
        a = []
        for i in pos_keyword.keys():
            if i in item:
                a.append(pos_keyword[i])
            else:
                a.append(0)
        pos_nn.append(a)
    for item in final_sentence:
        b = []
        for i in neg_keyword.keys():
            if i in item:
                b.append(neg_keyword[i])
            else:
                b.append(0)
        neg_nn.append(b)
    return pos_nn,neg_nn
pos_nn,neg_nn = LLR200(final_sentence,pos_keyword,neg_keyword)

#concatenate
final_input = np.concatenate((total_bert,final_top3VA,pos_nn,neg_nn),axis = 1)

In [ ]:
from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints
from keras import activations
from keras.engine.topology import Layer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Embedding, LSTM, Bidirectional
K.clear_session()

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionLayer(Layer):
    def __init__(self, attention_size=None, **kwargs):
        self.attention_size = attention_size
        super(AttentionLayer, self).__init__(**kwargs)
        
    def get_config(self):
        config = super().get_config()
        config['attention_size'] = self.attention_size
        return config
        
    def build(self, input_shape):
        assert len(input_shape) == 3
        
        self.time_steps = input_shape[1]
        hidden_size = input_shape[2]
        if self.attention_size is None:
            self.attention_size = hidden_size
            
        self.W = self.add_weight(name='att_weight', shape=(hidden_size, self.attention_size),
                                initializer='uniform', trainable=True)
        self.b = self.add_weight(name='att_bias', shape=(self.attention_size,),
                                initializer='uniform', trainable=True)
        self.V = self.add_weight(name='att_var', shape=(self.attention_size,),
                                initializer='uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)
    
    def call(self, inputs):
        self.V = K.reshape(self.V, (-1, 1))
        H = K.tanh(K.dot(inputs, self.W) + self.b)
        score = K.softmax(K.dot(H, self.V), axis=1)
        outputs = K.sum(score * inputs, axis=1)
        return outputs
    
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.utils import np_utils
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape, Flatten
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, BatchNormalization
from keras.layers.merge import concatenate
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

predicted= []
expected= []
cv = StratifiedKFold(n_splits=10,shuffle=True) 
cv.get_n_splits(final_input,label)
for train_idx, test_idx in cv.split(final_input,label):
    x_train = np.array(final_input)[train_idx]
    y_train = np.array(label)[train_idx]
    x_test = np.array(final_input)[test_idx]
    y_test = np.array(label)[test_idx] 
    x_train1 = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    x_test1 = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
    model = Sequential()
    model.add(Bidirectional(LSTM(128,return_sequences=True,input_shape=(1,final_input.shape[1]))))
    model.add(Activation('relu',name='attention_weights'))
    model.add(Bidirectional(LSTM(64,return_sequences=True)))
    model.add(Dropout(0.25))
    model.add(AttentionLayer())
    model.add(Dense(units = 64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(units = 32, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath="weights_waimai.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True,mode='max')
    callbacks_list = [checkpoint]
    print(model.summary())
    model.fit(x_train1, y_train, epochs=10, batch_size=32,callbacks=callbacks_list, verbose=1)
    expected.extend(y_test)
    predicted.extend(model.predict_classes(x_test1))